# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv('../WeatherPy/output_data/clean_city_data.csv')
city_data_df

,CityID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,butaritari,3.0707,172.7902,80.56,77,99,14.88,KI,1611978915
1,1,gazalkent,41.5581,69.7708,41.00,33,90,11.50,UZ,1611978915
2,2,hermanus,-34.4187,19.2345,62.01,90,63,1.01,ZA,1611978916
3,3,vaini,-21.2000,-175.2000,80.60,89,90,14.97,TO,1611978916
4,4,pareora,-44.4871,171.2132,63.00,57,24,5.99,NZ,1611978916
...,...,...,...,...,...,...,...,...,...,...
557,557,puerto narino,-3.7703,-70.3831,73.40,100,40,1.34,CO,1611979032
558,558,pran buri,12.3847,99.9098,84.20,58,91,9.22,TH,1611979032
559,559,steinbach,50.1667,8.5667,46.99,76,75,10.36,DE,1611979033
560,560,bafoulabe,13.8065,-10.8321,68.99,17,0,2.24,ML,1611979033


In [3]:
city_data_df.dtypes

CityID          int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [ ]:
#!jupyter nbextension enable --py --sys-prefix gmaps

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = city_data_df[['Lat', 'Lng']]
humidity = city_data_df[['Humidity']]
fig = gmaps.figure()

humidity = city_data_df['Humidity'].astype(float)
#rating = airport1_df["Airport Rating"].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrow_city_df = city_data_df.loc[(city_data_df['Max Temp'] < 80) \
                                 & (city_data_df['Max Temp'] > 70) \
                                 & (city_data_df['Wind Speed'] < 10) \
                                 & (city_data_df['Cloudiness'] == 0)].dropna()
narrow_city_df

,CityID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,bajil,15.0584,43.2851,70.72,81,0,4.74,YE,1611978918
31,31,caravelas,-17.7125,-39.2481,74.59,88,0,4.21,BR,1611978921
231,231,pisco,-13.7000,-76.2167,71.60,73,0,6.91,PE,1611978963
384,384,buenos aires,-34.6132,-58.3772,77.00,73,0,2.30,AR,1611978934
523,523,benjamin aceval,-24.9667,-57.5667,77.00,94,0,4.61,PY,1611979025


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrow_city_df[['City', 'Country', 'Lat', 'Lng']].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
12,bajil,YE,15.0584,43.2851,
31,caravelas,BR,-17.7125,-39.2481,
231,pisco,PE,-13.7000,-76.2167,
384,buenos aires,AR,-34.6132,-58.3772,
523,benjamin aceval,PY,-24.9667,-57.5667,


In [8]:
params = {
    "radius":5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    if index == 12:
        pprint(results)
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (KeyError, IndexError):
        print("Missing infomation...skipping")

hotel_df

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 15.05972680000001, 'lng': 43.2892609},
               'viewport': {'northeast': {'lat': 15.0611066302915,
                                          'lng': 43.29058198029149},
                            'southwest': {'lat': 15.0584086697085,
                                          'lng': 43.28788401970849}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
  'name': 'شبكة ماكس نت MAX WIFI',
  'place_id': 'ChIJn1GFKzn5BBYRyZas_Jxo7oQ',
  'plus_code': {'compound_code': '375Q+VP باجل, Yemen',
                'global_code': '7H75375Q+VP'},
  'reference': 'ChIJn1GFKzn5BBYRyZas_Jxo7oQ',
  'scope': 'GOOGLE',
  'types': ['lodging', 'point_of_interest', 'establishment'],
  'vicinity': ''},
 {'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 15.0636093, 'lng': 43.2841258},
               'viewport': {'northeast': {'lat': 15.0649235802915,
                           

,City,Country,Lat,Lng,Hotel Name
12,bajil,YE,15.0584,43.2851,شبكة ماكس نت MAX WIFI
31,caravelas,BR,-17.7125,-39.2481,Pousada dos Navegantes
231,pisco,PE,-13.7000,-76.2167,Hotel San Isidro Oficial
384,buenos aires,AR,-34.6132,-58.3772,Hotel NH Buenos Aires City
523,benjamin aceval,PY,-24.9667,-57.5667,Escuela Agrícola y Hotel Cerrito


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [9]:
locations = hotel_df[["Lat", "Lng"]]
print(type(locations))
locations

<class 'pandas.core.frame.DataFrame'>


,Lat,Lng
12,15.0584,43.2851
31,-17.7125,-39.2481
231,-13.7000,-76.2167
384,-34.6132,-58.3772
523,-24.9667,-57.5667


In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))